In [1]:
from LibriSpeechEntity import LibriSpeechEntity
import os
import librosa

/home/renyi/anaconda3/envs/cosyvoice2/lib/python3.8/site-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(


In [5]:
INPUTDIR = "/mnt/d/voicedata/test-clean/LibriSpeech/test-clean"

In [6]:
speakerchapter_dict = {}

In [10]:
AVG_DURATION = 600 # 10min = 600s
SAMPLE_RATE = 16000  # 16kHz (16000 samples per second), not seconds
OUT_DIR = '../datasets/LongSpeech'

In [7]:
# 计算每个人每个章节的duration
for speakers in os.listdir(INPUTDIR):
    speakerchapter_dict[speakers] = []
    for chapter in os.listdir(os.path.join(INPUTDIR, speakers)):
        duration = 0
        for wavs in os.listdir(os.path.join(INPUTDIR, speakers, chapter)):
            # 只处理flac文件
            if not wavs.endswith(".flac"):
                continue

            audio_path = os.path.join(INPUTDIR, speakers, chapter, wavs)
            # load audio
            audio, sr = librosa.load(audio_path, sr=None)
            duration += librosa.get_duration(y=audio, sr=sr)
        speakerchapter_dict[speakers].append((chapter, duration))




print(speakerchapter_dict)

{'1089': [('134686', 276.21006250000005), ('134691', 206.8500625)], '1188': [('133604', 491.93493750000016)], '121': [('121726', 79.09), ('123852', 76.64500000000001), ('123859', 93.15499999999999), ('127105', 231.69499999999996)], '1221': [('135766', 176.6), ('135767', 307.745)], '1284': [('1180', 227.9), ('1181', 146.97999999999996), ('134647', 114.5550625)], '1320': [('122612', 129.125), ('122617', 352.1499375000001)], '1580': [('141083', 257.0300625), ('141084', 227.25000000000003)], '1995': [('1826', 164.43506250000002), ('1836', 142.08499999999998), ('1837', 177.23506250000003)], '2094': [('142345', 485.64499999999975)], '2300': [('131720', 491.195)], '237': [('126133', 166.965), ('134493', 115.01500000000001), ('134500', 199.50493749999998)], '260': [('123286', 173.63), ('123288', 204.04506250000006), ('123440', 105.43999999999998)], '2830': [('3979', 92.1450625), ('3980', 390.30506249999996)], '2961': [('960', 281.9850625), ('961', 202.08999999999997)], '3570': [('5694', 223.76

In [13]:
# sort by duration

for speaker, chapters in speakerchapter_dict.items():
    # sort by duration in descending order
    chapters.sort(key=lambda x: x[1], reverse=True)
    speakerchapter_dict[speaker] = chapters

# sort speakers by the total duration of their chapters
speakerchapter_dict = dict(sorted(speakerchapter_dict.items(), key=lambda item: sum(duration for _, duration in item[1]), reverse=True))

speakerchapter_dict

{'7127': [('75947', 263.26493750000003), ('75946', 235.74)],
 '5639': [('40744', 496.68993750000004)],
 '672': [('122797', 496.1649999999999)],
 '8230': [('279154', 494.80000000000007)],
 '8224': [('274381', 331.1649999999999), ('274384', 163.4)],
 '6829': [('68769', 247.80499999999998), ('68771', 246.655)],
 '4992': [('41797', 175.83499999999998),
  ('41806', 172.08),
  ('23283', 144.42499999999998)],
 '1188': [('133604', 491.93493750000016)],
 '2300': [('131720', 491.195)],
 '7729': [('102255', 489.94506250000006)],
 '1284': [('1180', 227.9),
  ('1181', 146.97999999999996),
  ('134647', 114.5550625)],
 '4970': [('29095', 278.30999999999995), ('29093', 210.885)],
 '7021': [('85628', 188.22500000000002),
  ('79730', 123.6),
  ('79740', 122.05000000000001),
  ('79759', 54.614999999999995)],
 '4077': [('13751', 273.26000000000005), ('13754', 215.21493750000005)],
 '5105': [('28241', 190.12999999999997),
  ('28240', 178.54506250000003),
  ('28233', 118.78500000000001)],
 '2094': [('142345

In [6]:
def stream_librispeech_grouped_all(target_len_sec=30.0, sampling_rate=16000):
    ds_dict = load_dataset(
        "openslr/librispeech_asr",
        "all",
        streaming=True,
    )

    def every_sample():
        for split in ds_dict.values():
            split = split.cast_column("audio",
                                       Audio(sampling_rate=sampling_rate))
            for sample in split:
                yield sample

    buffer, buf_dur, current_key = [], 0.0, None
    for s in every_sample():
        key = (s["speaker_id"], s["chapter_id"])
        wav = s["audio"]["array"]
        dur = len(wav) / sampling_rate

        if current_key is None:
            current_key = key

        # 遇到新卷或时长超阈值 → 把缓冲区吐出来
        if key != current_key or buf_dur + dur >= target_len_sec:
            if buffer:
                yield {
                    "speaker_id": current_key[0],
                    "chapter_id": current_key[1],
                    "audio": np.concatenate(buffer),
                    "length_sec": buf_dur,
                }
            buffer, buf_dur, current_key = [], 0.0, key

        buffer.append(wav); buf_dur += dur

    # 文件结尾还有残余
    if buffer:
        yield {
            "speaker_id": current_key[0],
            "chapter_id": current_key[1],
            "audio": np.concatenate(buffer),
            "length_sec": buf_dur,
        }
